How many articles mention contact tracing (vs. entire coronavirus corpus) per country?

In [2]:
import pandas as pd
import pickle

In [4]:
us = pd.read_pickle("us/data/us_proc.pkl")
kor = pd.read_pickle("korea/data/korean_proc.pkl")
taiwan = pd.read_csv("taiwan/data/text_proc.csv")
hk_eng = pd.read_csv("hong kong/data/hk_eng_proc.csv")
hk_ch = pd.read_csv("hong kong/data/hk_ch_proc.csv")

In [27]:
us_ct = len(us[us.text_proc.str.contains("contact tracing|contact-tracing|contact tracking|contact-tracking")])
kor_ct = len(kor[kor.text_proc.str.contains("역학 조사|확진자 추적|컨택트 트레이싱|추적 조사|접촉자 추적|동선 추적")])
taiwan_ct = len(taiwan[taiwan.text_proc.str.contains("接觸 追蹤|持續 追蹤|接觸者 追蹤|追蹤 接觸者")])
hk_ch_ct = len(hk_ch[hk_ch.text_proc.str.contains("接觸 追蹤|接觸者 追蹤|追蹤 接觸者|流行病學 調查")])
hk_eng_ct = len(hk_eng[hk_eng.text_proc.str.contains("contact tracing|contact-tracing|contact tracking|contact-tracking")])

In [30]:
ct_articles = [
    [us_ct, len(us)], [kor_ct, len(kor)], [taiwan_ct, len(taiwan)], 
    [hk_ch_ct, len(hk_ch)], [hk_eng_ct, len(hk_eng)]
]

In [44]:
df = pd.DataFrame(data=ct_articles,
             index=['us', 'korea', 'taiwan', 'hong kong chinese', 'hong kong english'], 
             columns=['contact tracing articles', 'total articles'])

In [48]:
df.loc[:, 'percentage'] = df['contact tracing articles']/df['total articles'] * 100

In [49]:
df

,contact tracing articles,total articles,percentage
us,1836,51541,3.562213
korea,2732,51250,5.330732
taiwan,344,84900,0.405183
hong kong chinese,135,22525,0.599334
hong kong english,385,4822,7.984239
